In [ ]:
import socket

HOST = '127.0.0.1'
PORT = 4326



with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as serverSocket:
    # allows the socket's methods to be non-blocking
    serverSocket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
    print(serverSocket)


    # local address and port binding on server
    serverSocket.bind((HOST, PORT))

    # socket connection will only listen for one connection
    serverSocket.listen(5)

    # set socket to be non-blocking
    serverSocket.setblocking(False)

    print("waiting for incomming connections on ", serverSocket.getsockname())

    # TODO: implement the select method

    # `accept` is an async method
    acceptConnection(serverSocket)

    with clientSocket:
        print("accepted connection from: ", clientAddress)

        while True:
            # it is not guaranteed that all data transfered through tcp will arrive at its destination
            # `recv` is an async function
            # TODO: implement error handing (timeout event)
            data = clientSocket.recv(1024)

            # check that data was received
            print("received %d bytes from the client" % len(data))

            if not data:
                break

            # TODO: implement error handing (timeout event)
            len = clientSocket.send(data) # sends the same data back
            print("sent %d bytes back to the client server" % len)

In [ ]:
# bytes formatting

import struct

var = struct.pack('!s', b"client message\r\n")
print(var)

In [3]:
import selectors

sel = selectors.DefaultSelectors()

class ClientData:
    def __init__(self, addr):
        self.addr = addr
        self.inb = b''
        self.outb = b''

# use instead of the accept
def acceptConnection(sock):
    conn, addr = sock.accept()
    print('accepted connection from %s' % addr)
    
    conn.setblocking(False)
    
    # concurrent data proccesing
    data = ClientData(addr)
    events = selectors.EVENT_READ | selectors.EVENT_WRITE
    sel.register(conn, events, data = data)
    
def serviceConnection(key, mask):
    sock = key.fileobj
    data = key.data

    if mask & selectors.EVENT_READ:
        recv_data = sock.recv(1024)
        if recv_data:
            data.outb += recv_data
        else:
            print('closing connection to', data.addr)
            sel.unregister(sock)
            sock.close()
    
    if mask & selectors.EVENT_WRITE:
        if data.outb:
            print('echoing', repr(data.outb), 'to', data.addr)
            sent = sock.send(data.outb)
            data.outb = data.outb[sent:]

In [ ]:
# connections concurrency options:
# - threading - race conditions increase complexity
# - select() - explicit flows, no race conditions

import select


sel = selectors.DefaultSelectors()
sel.register(serverSocket, selectors.EVENT_READ, data = None)

while True:
    events = sel.select(timeout = 10)

    for key, mask in events:
        if key.data is None:
            acceptConnection(key.fileobt)
        else:
            serviceConnection(key, mask)